In [1]:
# Smoke test for Plaid /transactions/sync.
# Prints added/modified/removed counts and the next_cursor for each Item.
# Does not write cursors or any files.

import os, json, sys
from pathlib import Path
from dotenv import load_dotenv

from plaid.configuration import Configuration, Environment
from plaid.api import plaid_api
from plaid.model.transactions_sync_request import TransactionsSyncRequest
from plaid.model.transactions_sync_request_options import TransactionsSyncRequestOptions
from plaid.exceptions import ApiException

REPO = Path(__file__).resolve().parents[1]
CONFIG_DIR = REPO / "config"
ITEMS_JSON = CONFIG_DIR / "plaid_items.json"
CURSORS_JSON = CONFIG_DIR / "plaid_cursors.json"  # optional

# 1) load env
load_dotenv(REPO / ".env")
CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
SECRET = os.getenv("PLAID_SECRET")
ENV = os.getenv("PLAID_ENV", "sandbox").lower()

if not CLIENT_ID or not SECRET or ENV != "sandbox":
    print("Error: .env must have PLAID_CLIENT_ID, PLAID_SECRET and PLAID_ENV=sandbox", file=sys.stderr)
    sys.exit(1)

# 2) plaid api client
configuration = Configuration(
    host=Environment.Sandbox,
    api_key={"clientId": CLIENT_ID, "secret": SECRET},
)
api_client = plaid_api.PlaidApi(configuration)

# 3) load items
if not ITEMS_JSON.exists():
    print(f"Missing {ITEMS_JSON}. Run create_sandbox_items.py first.", file=sys.stderr)
    sys.exit(1)

with ITEMS_JSON.open("r", encoding="utf-8") as f:
    items = (json.load(f) or {}).get("items", [])

if not items:
    print("No items found in plaid_items.json", file=sys.stderr)
    sys.exit(1)

# 4) load cursors if present (ok if missing)
cursors = {}
if CURSORS_JSON.exists():
    try:
        cursors = (json.load(CURSORS_JSON.open("r", encoding="utf-8")) or {}).get("transactions", {})
    except Exception:
        cursors = {}

print(f"PLAID ENV: {ENV} | Items: {len(items)}")
print("-" * 60)

# 5) call /transactions/sync for each item
for it in items:
    item_id = it["item_id"]
    token = it["access_token"]
    cursor = cursors.get(item_id) or None

    print(f"[{item_id}] starting cursor: {cursor}")
    added = modified = removed = 0
    latest_cursor = cursor

    try:
        has_more = True
        while has_more:
            req = TransactionsSyncRequest(
                access_token=token,
                cursor=latest_cursor,
                count=500,
                options=TransactionsSyncRequestOptions(include_personal_finance_category=False),
            )
            resp = api_client.transactions_sync(req)
            added += len(resp["added"])
            modified += len(resp["modified"])
            removed += len(resp["removed"])
            latest_cursor = resp["next_cursor"]
            has_more = resp["has_more"]

        print(f"[{item_id}] added={added} modified={modified} removed={removed}")
        print(f"[{item_id}] next_cursor (NOT saved): {latest_cursor}")
        print("-" * 60)

    except ApiException as e:
        print(f"[{item_id}] Plaid API error: {e}", file=sys.stderr)
        sys.exit(1)



NameError: name '__file__' is not defined